In [2]:
AGENT_NAME = '20 bin PPO 500 results/default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
ATTACK = 'myBB'
TRIAL_PATH ='optuna/bb hyperparameters 1'

In [3]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from KBMproject.mybb import BrendelBethgeAttack as BBA

import pandas as pd
import numpy as np
import os

import KBMproject.utilities as utils

Define RL agent

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [5]:
from art.estimators.classification import PyTorchClassifier as classifier
from torch.nn import CrossEntropyLoss
victim_policy = classifier(
    model=utils.extract_actor(agent),
    loss=CrossEntropyLoss(), 
    nb_classes=agent.action_space[0].n,
    input_shape=agent.observation_space.shape,
    device_type='gpu',
    clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max()) #min and max values of each feature, brendle bethge attack only supports floats values and not array
    )

In [6]:
mask_time = np.ones(agent.observation_space.shape)
mask_time[:6] = 0

In [8]:
df_baseline = pd.concat([pd.read_csv('20 bin PPO 500 results/baseline_obs.csv', index_col=0,dtype='float32'),
                        pd.read_csv('20 bin PPO 500 results/baseline_obs_a_confidence.csv', index_col=0)], axis='columns')

Below, for each action we select the sample with the highest logit softmax/confidence

In [9]:
df_max_confidence_samples = df_baseline.loc[df_baseline.groupby('action')['confidence'].idxmax()]
df_max_confidence_samples = df_max_confidence_samples.drop(columns='confidence')
df_max_confidence_samples = df_max_confidence_samples.set_index('action')

In [10]:
dict_samples = df_max_confidence_samples.T.to_dict()

In [12]:
samples = pd.read_csv('20 bin PPO 500 results/bb results/optimal myBB observations.csv', index_col=0,dtype='float32')

Save clean obs from optimal bb attack in trial folder

In [13]:
inputs = samples.to_numpy()[:-1]
np.savetxt(f"{TRIAL_PATH}/inputs.csv", inputs, delimiter=",")

Generate advserarial actions and save in trial folder

In [15]:
targets = np.argmax(victim_policy.predict(pd.read_csv('20 bin PPO 500 results/bb results/optimal myBB adversarial observations.csv',
                                            index_col=0,
                                            dtype='float32')
                                            ), axis=1)
np.savetxt(f"{TRIAL_PATH}/targets.csv", targets, delimiter=",")

generate advsersarial initiializations and save in trial folder

In [ ]:
starts = np.array([np.array(list(
    dict_samples[a].values())).astype('float32') if a in dict_samples else np.zeros(agent.observation_space.shape, 'float32') for a in targets]) 
#there is no example for action 18

In [ ]:
np.savetxt(f"{TRIAL_PATH}/starts.csv", starts, delimiter=",")

Save victim policy in trial folder

There's no way to laod it as an ART classifier

measure how long one trial would take/how long it takes to generate adv samples for an episode:

should be about 20 minutes, but that' only with one running